## import dependencies

In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

## data collection and preprocessing

In [4]:
#load data
raw_mail_data= pd.read_csv("mail_data.csv")

In [5]:
raw_mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
raw_mail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
raw_mail_data.shape

(5572, 2)

In [12]:
 #replaces all missing values (NaN, None) in with empty strings ('').
mail_data= raw_mail_data.where((pd.notnull(raw_mail_data)),'')

# Label Encoding

In [15]:
# label spam mail as 0;  ham mail as 1;
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

spam = 0

ham = 1

In [16]:
mail_data.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
# separating the data as texts and label

X = mail_data['Message'] # feature

y = mail_data['Category'] # target

# Split the data into training and test data

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [23]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)

(5572,)
(4457,)
(1115,)
(1115,)


# Feature Extraction

In [24]:
# transform the text data to feature vectors that can be used as input to the Logistic regression

feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [25]:
print(X_train)

3075                  Don know. I did't msg him recently.
1787    Do you know why god created gap between your f...
1614                         Thnx dude. u guys out 2nite?
4304                                      Yup i'm free...
3266    44 7732584351, Do you want a New Nokia 3510i c...
                              ...                        
789     5 Free Top Polyphonic Tones call 087018728737,...
968     What do u want when i come back?.a beautiful n...
1667    Guess who spent all last night phasing in and ...
3321    Eh sorry leh... I din c ur msg. Not sad alread...
1688    Free Top ringtone -sub to weekly ringtone-get ...
Name: Message, Length: 4457, dtype: object


In [ ]:
print(X_train_features)

# Train a LogisticRegression Model

In [27]:
model = LogisticRegression()

In [28]:
# training the Logistic Regression model with the training data
model.fit(X_train_features, y_train)

LogisticRegression()

# Evaluating the trained model

In [29]:
prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)

In [30]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9676912721561588


In [31]:
# prediction on test data

prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)

In [32]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.9668161434977578


In [34]:
print(classification_report(y_test, prediction_on_test_data))

              precision    recall  f1-score   support

           0       1.00      0.76      0.86       155
           1       0.96      1.00      0.98       960

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



# Classification report explanation
## Per-Class Performance

Class 0 (Spam)

- Precision = 1.00 → Out of all emails predicted as spam, 100% were actually spam. That means the model never falsely labelled a ham email as spam.

- Recall = 0.76 → Out of all the actual spam emails, it correctly identified 76%. But it missed 24% spam emails, labelling them as ham.

- F1-score = 0.86 → Balance between precision and recall for spam detection. Pretty good, but recall could improve.

Class 1 (Ham)

- Precision = 0.96 → Out of all emails predicted as ham, 96% were actually ham. A few spam emails got wrongly classified as ham (false negatives).

- Recall = 1.00 → Out of all the actual ham emails, 100% were correctly classified. The model never missed a ham email.

- F1-score = 0.98 → Very strong balance for ham detection.

## Overall Performance

- Accuracy = 0.97 → The model is correct 97% of the time overall. Very strong.

- Macro Avg (0.98 precision, 0.88 recall, 0.92 f1) → Macro average treats spam and ham equally. Since recall for spam is lower (0.76), it pulls the macro recall down.

- Weighted Avg (0.97 precision, 0.97 recall, 0.96 f1) → Weighted average accounts for class imbalance (ham is much more frequent than spam). That’s why weighted scores are close to accuracy.

## Key Insight

- The model is excellent at catching ham (non-spam) messages and avoids flagging them wrongly as spam.

- The weakness: It misses some spam emails (recall 0.76) — meaning some spam slips through as ham.

` NOTE`
In spam detection, recall for spam (class 0) is usually more important than precision, because missing spam could mean users receive unwanted or dangerous emails.

# Building a predictive system

In [51]:
input_mail = ["“Important: your account will be closed, verify here."]
# convert text to feature vectors
input_data_features = feature_extraction.transform(input_mail)

# making prediction

prediction = model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Ham mail')

else:
  print('Spam mail')

[1]
Ham mail
